In [5]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics import precision_score, confusion_matrix
import re

In [2]:
def compute_metrics(file):

    threshold = 3
    model_num = re.findall(r'\d+\.?\d*', file)[0]
    
    df = pd.read_csv(file)
    
    y_true = df['y_true']
    y_pred = df['y_pred']
    y_pred_round = y_pred.map(lambda x: np.around(x)).to_numpy()
    
    precision = precision_score(y_true, y_pred_round, zero_division = 0)
    cm = confusion_matrix(y_true.to_numpy(), y_pred_round)
    num_TP = cm[1,1]
    num_FP = cm[0,1]
    
    idx = (y_pred_round == y_true)
    correct_pred_confidence_avg = (1 - abs(y_true[idx] - y_pred[idx])).mean()
    correct_pred_confidence_std = (1 - abs(y_true[idx] - y_pred[idx])).std()
    correct_pred_confidence = str(correct_pred_confidence_avg) + " ± " + str(correct_pred_confidence_std)

    idx = ((y_pred_round == 1) & (y_true == 0))
    FP = y_pred[idx].reset_index(drop=True).sort_values(ascending=False)[threshold]
    # FP_confidence_avg = abs(y_true[idx] - y_pred[idx]).mean()
    # FP_confidence_std = abs(y_true[idx] - y_pred[idx]).std()
    # FP_confidence = str(FP_confidence_avg) + " ± " + str(FP_confidence_std)
    
    
    # idx = ((y_pred_round == 1) & (y_true == 1))
    # TP_confidence_avg = (1 - abs(y_true[idx] - y_pred[idx])).mean()
    # TP_confidence_std = (1 - abs(y_true[idx] - y_pred[idx])).std()
    # TP_confidence = str(TP_confidence_avg) + " ± " + str(TP_confidence_std)

    # return [model_num, precision, num_TP, correct_pred_confidence_avg, correct_pred_confidence_std,
    #         FP_confidence_avg, FP_confidence_std, TP_confidence_avg, TP_confidence_std]
    return [model_num, precision, num_TP, num_FP, correct_pred_confidence, FP_confidence, TP_confidence]

In [3]:
os.getcwd()

'/Users/gabrielgreenstein/Desktop/Research/Appel Lab/GNNs/antimicrobial_GNN/model_performance_analysis'

In [4]:
directory = '/Users/gabrielgreenstein/Downloads/MPNN/hyperparameter_optimization'

# data = [['model_num', 'precision', 'num_TP', 'correct_pred_confidence_avg', 'correct_pred_confidence_std',
#          'FP_confidence_avg', 'FP_confidence_std', 'TP_confidence_avg', 'TP_confidence_std']]
data = [['model_num', 'precision', 'num_TP', 'num_FP', 'correct_pred_confidence', 'FP_confidence', 'TP_confidence']]

for model_folder in os.listdir(directory):
    model_folder_path = os.path.join(directory, model_folder)
    if os.path.isdir(model_folder_path) and model_folder.startswith('model_'):
        
        val_model_folder = os.path.join(model_folder_path, 'val_model_on_infer_set')
        
        if os.path.isdir(val_model_folder):
            
            results_file = os.path.join(val_model_folder, 'results.txt')
            
            if os.path.isfile(results_file):
                data.append(compute_metrics(results_file))
            else:
                print(f"'results.txt' not found in {val_model_folder}")
        else:
            print(f"'val_model_on_infer_set' folder not found in {model_folder_path}")


NameError: name 'FP_confidence' is not defined

In [223]:
df = pd.DataFrame(data[1:], columns=data[0])
df = df[df['precision'] >= 0.85]
df.sort_values(by=['precision'], inplace=True, ascending=False)
df.reset_index(drop=True, inplace=True)
df.fillna(0)
# df['hit_cutoff'] = df.apply(lambda row: min([row['TP_FN_confidence_avg'] - row['TP_FN_confidence_std'],
#                                              [row['FP_confidence_avg'] + row['FP_confidence_std']]]),
#                             axis=1)

,model_num,precision,num_TP,num_FP,correct_pred_confidence,FP_confidence,TP_confidence
0,4,1.000000,393,0,0.9949527556761949 ± 0.03833265784511841,nan ± nan,0.9719397251235923 ± 0.08697965443536548
1,96,1.000000,1,0,0.9999511840275961 ± 0.002071661528892238,nan ± nan,0.9120824337005616 ± nan
2,59,1.000000,516,0,0.9905869959273945 ± 0.05006327048715268,nan ± nan,0.9577509686928387 ± 0.09937977300760242
3,61,1.000000,21,0,0.9996223324801339 ± 0.007116956986142751,nan ± nan,0.9672509658904302 ± 0.05912791640759164
4,83,1.000000,806,0,0.993520425383295 ± 0.04093457784372255,nan ± nan,0.9792268547615401 ± 0.07151685455507069
5,34,1.000000,12,0,0.9989684011353656 ± 0.01747289343389826,nan ± nan,0.8442290723323822 ± 0.15481815479911812
6,56,1.000000,2,0,0.9998924752715954 ± 0.004564230492866443,nan ± nan,0.903120219707489 ± 0.13699723526659968
7,45,1.000000,3,0,0.9998880812406802 ± 0.0032248628544584088,nan ± nan,0.9522722562154134 ± 0.06526165497297141
8,37,1.000000,160,0,0.9949267946867513 ± 0.03654372424589149,nan ± nan,0.9378532372415066 ± 0.11350960607852734
9,99,1.000000,17,0,0.9984691436287616 ± 0.02288562279444848,nan ± nan,0.8363784937297597 ± 0.17682395317549843


In [314]:
file = './generic_val_on_infer/Weave/hyperparameter_optimization/model_57/val_model_on_infer_set/results.txt'

In [321]:
a = pd.read_csv(file)
a['adjusted_y_pred'] = a['y_pred'].map(lambda x: 1 if x >= 0.99999 else 0)

In [322]:
cm = confusion_matrix(a['y_true'], a['adjusted_y_pred'].to_numpy())
cm

array([[1767,   33],
       [1308, 1492]])

In [323]:
s = a[(a['adjusted_y_pred'] == 1) & (a['y_true'] == 0)]['y_pred'].reset_index(drop=True).sort_values(ascending=False)
s.loc[s < 1].iloc[0]

0.9999998807907104